In [44]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cPickle as pickle
import os

%matplotlib inline

In [95]:
# Build buffer database
# > cd /home/data/alfa/

# > scp abc3:/databk/datProcessing/output/ALFAbuffers.pkl .
baseBufferPklFile = '/home/griffin/data/alfa/ALFAbuffers.pkl'

# load baseBufferPkl
df = pd.read_pickle(baseBufferPklFile)

# create a predicted label column with 'unlabelled' label
df = df.assign(predictLabel=-1)

In [114]:
#print df.columns

# metadata and features pickles
baseDedispDirs = ['/home/griffin/data/alfa/dedisp/snr14_dm50/',
                  '/home/griffin/data/alfa/dedisp/snr11-14_dm50/']

# extract features from buffer meta-data pickle files and add to the main dataframe
# ['maxMJD', 'dat', 'DMmin', 'DMmedian', 'pctZero', 'pctZeroDeriv', 'filterbank', 'DMmean', 'overflows',
# 'nEvents', 'maxSNR', 'globalTimeStats', 'windDedispTimeStats', 'windTimeStats', 'RA', 'MJD0', 'longestRun',
# 'Dec', 'DMmax', 'pixels', 'globalDedispTimeStats']
for dDir in baseDedispDirs:
    for subDir in os.listdir(dDir):
        if os.path.isdir(dDir + '/' + subDir):
            metaPklFns = glob.glob(dDir + subDir + '/*.meta.pkl')
            
            #print pickle.load(open(metaPklFns[0], 'rb')).keys()
            for mIdx, metaPkl in enumerate(metaPklFns):
                baseMetaFn = os.path.basename(metaPkl)
                bufID = int(baseMetaFn.split('.')[1].split('buffer')[-1])
                metaDict = pickle.load(open(metaPkl, 'rb'))
                df.loc[(df['datfile']==metaDict['dat']) & (df['Buffer']==bufID), 'filterbank'] = metaDict['filterbank']


Index([u'datfile', u'Beam', u'TSID', u'Buffer', u'MJDstart', u'bestDM',
       u'bestSNR', u'BinFactor', u'Events', u'DMmax', u'DMmin', u'DMmean',
       u'DMmedian', u'DMstd', u'SNRmean', u'SNRmedian', u'SNRstd', u'MJDmax',
       u'MJDmin', u'MJDstd', u'MJDmean', u'MJDmedian', u'Label',
       u'predictLabel', u'filterbank'],
      dtype='object')
['maxMJD', 'dat', 'DMmin', 'DMmedian', 'pctZero', 'pctZeroDeriv', 'filterbank', 'DMmean', 'overflows', 'nEvents', 'maxSNR', 'globalTimeStats', 'windDedispTimeStats', 'windTimeStats', 'RA', 'MJD0', 'longestRun', 'Dec', 'DMmax', 'pixels', 'globalDedispTimeStats']
['maxMJD', 'dat', 'DMmin', 'DMmedian', 'pctZero', 'pctZeroDeriv', 'filterbank', 'DMmean', 'overflows', 'nEvents', 'maxSNR', 'globalTimeStats', 'windDedispTimeStats', 'windTimeStats', 'RA', 'MJD0', 'longestRun', 'Dec', 'DMmax', 'pixels', 'globalDedispTimeStats']
['maxMJD', 'dat', 'DMmin', 'DMmedian', 'pctZero', 'pctZeroDeriv', 'filterbank', 'DMmean', 'overflows', 'nEvents', 'maxSNR', 

IndexError: list index out of range

In [111]:
# output of labelImg2.py
labelPKlFiles = glob.glob('/home/griffin/data/alfa/dedisp/snr14_dm50/labels/snr*.pkl')

#add assigned labels to main dataframe
for lPkl in labelPKlFiles:
    labelDict = pickle.load(open(lPkl, 'rb'))
    for key,val in labelDict.iteritems():
        fbFN = key.split('buffer')[0] + 'fil'
        bufID = int(key.split('.')[1].split('buffer')[-1])
        df.loc[(df['filterbank']==fbFN) & (df['Buffer']==bufID), 'Label'] = val


                             datfile  Beam              TSID  Buffer  \
0      Beam0_dm_D20150624T020452.dat     0  D20150624T020452       1   
1      Beam0_dm_D20150624T020452.dat     0  D20150624T020452       2   
2      Beam0_dm_D20150624T020452.dat     0  D20150624T020452       3   
3      Beam0_dm_D20150624T020452.dat     0  D20150624T020452       4   
4      Beam0_dm_D20150624T020452.dat     0  D20150624T020452       5   
5      Beam0_dm_D20150624T020452.dat     0  D20150624T020452       6   
6      Beam0_dm_D20150624T020452.dat     0  D20150624T020452       7   
7      Beam0_dm_D20150624T020452.dat     0  D20150624T020452       8   
8      Beam0_dm_D20150624T020452.dat     0  D20150624T020452       9   
9      Beam0_dm_D20150624T020452.dat     0  D20150624T020452      10   
10     Beam0_dm_D20150624T020452.dat     0  D20150624T020452      11   
11     Beam0_dm_D20150624T020452.dat     0  D20150624T020452      12   
12     Beam0_dm_D20150624T020452.dat     0  D20150624T020452    

In [ ]:
# add subset of features
# build classifier